In [1]:
!pip install alpha_vantage

In [2]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries


class ScriptData:
    def __init__(self):
        self.data = {}

    def fetch_intraday_data(self, script):
        ts = TimeSeries(key='FYRQ9G4D1SERFT1V', output_format='pandas')
        data, _ = ts.get_intraday(symbol=script, interval='1min')
        self.data[script] = data

    def convert_intraday_data(self, script):
        if script not in self.data:
            raise ValueError(f"{script} data not found.")
        data = self.data[script]
        df = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        for index, row in data.iterrows():
            ts = pd.Timestamp(index)
            row_data = [ts, row['1. open'], row['2. high'], row['3. low'], row['4. close'], row['5. volume']]
            df.loc[len(df)] = row_data
        return df

    def __getitem__(self, key):
        if key not in self.data:
            raise KeyError(f"{key} not found.")
        return self.convert_intraday_data(key)

    def __setitem__(self, key, value):
        raise NotImplementedError("Setting data using setitem method is not implemented.")

    def __contains__(self, key):
        return key in self.data.keys() or any(key in df.index.get_level_values(0) for df in self.data.values())




In [3]:
script_data = ScriptData()

In [4]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,97.10,97.11,97.10,97.10,1470.0
1,2023-02-15 19:59:00,97.12,97.12,97.12,97.12,684.0
2,2023-02-15 19:58:00,97.12,97.12,97.12,97.12,1298.0
3,2023-02-15 19:57:00,97.10,97.12,97.10,97.12,1047.0
4,2023-02-15 19:56:00,97.12,97.12,97.10,97.10,806.0
...,...,...,...,...,...,...
95,2023-02-15 18:01:00,96.90,96.95,96.89,96.94,6472.0
96,2023-02-15 18:00:00,96.90,96.90,96.90,96.90,585.0
97,2023-02-15 17:59:00,96.90,96.91,96.90,96.91,760.0
98,2023-02-15 17:58:00,96.93,96.93,96.93,96.93,122.0


In [5]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data("AAPL")
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.51,155.5100,155.48,155.4800,7147.0
1,2023-02-15 19:59:00,155.50,155.5100,155.49,155.5100,3725.0
2,2023-02-15 19:58:00,155.48,155.5000,155.48,155.4901,1386.0
3,2023-02-15 19:57:00,155.49,155.5000,155.49,155.4900,1288.0
4,2023-02-15 19:56:00,155.48,155.4800,155.48,155.4800,690.0
...,...,...,...,...,...,...
95,2023-02-15 17:12:00,155.29,155.2900,155.29,155.2900,120.0
96,2023-02-15 17:10:00,155.29,155.2900,155.29,155.2900,359.0
97,2023-02-15 17:08:00,155.29,155.2900,155.29,155.2900,275.0
98,2023-02-15 17:07:00,155.30,155.3085,155.30,155.3085,283.0


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    
    df['indicator'] = df.close.rolling(window=timeperiod).mean()    
    indicator=df[['timestamp', 'indicator']]
    return indicator

In [10]:
indicator1(script_data['GOOGL'], timeperiod=5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:59:00,NaN
2,2023-02-15 19:58:00,NaN
3,2023-02-15 19:57:00,NaN
4,2023-02-15 19:56:00,97.112
...,...,...
95,2023-02-15 18:01:00,96.954
96,2023-02-15 18:00:00,96.940
97,2023-02-15 17:59:00,96.930
98,2023-02-15 17:58:00,96.928


In [11]:
indicator1(script_data['AAPL'], timeperiod=5)

,timestamp,indicator
0,2023-02-15 20:00:00,NaN
1,2023-02-15 19:59:00,NaN
2,2023-02-15 19:58:00,NaN
3,2023-02-15 19:57:00,NaN
4,2023-02-15 19:56:00,155.49002
...,...,...
95,2023-02-15 17:12:00,155.27200
96,2023-02-15 17:10:00,155.28000
97,2023-02-15 17:08:00,155.28800
98,2023-02-15 17:07:00,155.29370


In [12]:
class Strategy:
    def __init__(self, script_name):
        self.script_name = script_name
        
    
    def get_script_data(self):
        script_data = ScriptData()
        self.df=script_data.fetch_intraday_data(self.script_name)
        self.df = script_data.convert_intraday_data(self.script_name)
        self.close_data=self.df['close']
        indi_data=self.indicator_data = indicator1(self.df, timeperiod=5)
        self.indicator_data=indi_data['indicator']
        #return self.indicator_data
    

    def get_signals(self):
        signals = pd.DataFrame({'timestamp': pd.to_datetime(self.df['timestamp'])})
        signals['signal'] = 'NO_SIGNAL'
        for i in range(1, len(self.close_data)):
            
            if self.indicator_data[i] > self.close_data[i] and self.indicator_data[i-1] < self.close_data[i-1]:
                    signals.loc[i, 'signal'] = 'BUY'
            elif self.indicator_data[i] < self.close_data[i] and self.indicator_data[i-1] > self.close_data[i-1]:
                    signals.loc[i, 'signal'] = 'SELL'
        
        signals.loc[self.indicator_data > self.close_data, 'signal'] = 'BUY'
        signals.loc[self.indicator_data < self.close_data, 'signal'] = 'SELL'
        signals = signals[signals['signal'].isin(['BUY', 'SELL'])]
        print(signals)
        

In [13]:
strategy = Strategy('NVDA')

In [14]:
strategy.get_script_data()

In [15]:
strategy.get_signals()

             timestamp signal
4  2023-02-15 19:54:00   SELL
5  2023-02-15 19:53:00   SELL
6  2023-02-15 19:52:00    BUY
7  2023-02-15 19:51:00    BUY
8  2023-02-15 19:50:00    BUY
..                 ...    ...
95 2023-02-15 17:24:00    BUY
96 2023-02-15 17:23:00    BUY
97 2023-02-15 17:21:00    BUY
98 2023-02-15 17:20:00    BUY
99 2023-02-15 17:15:00    BUY

[95 rows x 2 columns]


In [20]:
import plotly.graph_objects as go
import plotly.express as px

symbol='AAPL'
data=script_data[symbol]
open_data = data['open']
high_data=data['high']
low_data =data['low']
close_data =data['close']
dates=data['timestamp']
indicator=indicator1(script_data[symbol], timeperiod=5)

fig = go.Figure(data=[go.Candlestick(x=dates,
                       open=open_data, high=high_data,
                       low=low_data, close=close_data)])

fig.show()
ind = px.line(indicator["indicator"], x=dates)

ind.show()

